# 本文件说明
- 数据库里导出数据，本地模型、线上模型测试

# 基本设置

In [2]:
import numpy as np
import pandas as pd

import os

import requests,json
from sklearn.externals import joblib

import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [40]:
from toolkits.setup.date_time import get_day_list
from toolkits.setup import specific_func

## 一些函数

In [5]:
def get_server_res_yjh(data, url, col_name):
    '''
    服务器接口测试程序
    传入 dict, 传出 DataFrame
    '''
    # data = {'record':[{'id':0,'title':'ss','content':'zzz'},]}
    # data = {"record":marked_human_data.iloc[:5,:3].to_dict(orient = 'records')}
    # url "http://47.93.77.19:10000/correlation_negative"
    headers={'content-type':'application/json'}
    result = requests.post(url,
                      data = json.dumps(data),
                      headers=headers, allow_redirects=True)
    # print(result.text)
    json_data = json.loads(result.text)
    parse_data = []
#     elapsed_time = json_data['elapsed_time']
    for i in range(len(json_data['docs'])):
        parse_data.append([json_data['docs'][i]['id'],
                          json_data['docs'][i][col_name]])
    parse_data = pd.DataFrame(parse_data, columns = ['id', col_name])    
    return parse_data #, elapsed_time

In [6]:
def get_serve_data_yjh(day_list, sql_one_day, url, col_name, save_filename):    
    chunksize = 1000
    for day_select in day_list:
        print('-- day_select: ', day_select)
        mysql_data = pd.read_sql(eval(sql_one_day), engine, chunksize= chunksize)
        num = 1
        combined_data = pd.DataFrame()
        for tmp_data in mysql_data:  
            print('---- loop num: ', num, 'tmp_data: ', tmp_data.shape)
            data = {"record":tmp_data.loc[:,['id', 'title' ,'content']].to_dict(orient = 'records')}
            parse_data = get_server_res_yjh(data, url, col_name)

            parse_data.columns = ['id', 'predict_label']
            
            parse_data['label'] = ''
            combined_tmp = pd.merge(parse_data, tmp_data, on = 'id', how = 'inner')
            combined_data = pd.concat([combined_tmp, combined_data])

        combined_data['predict_label'] = combined_data['predict_label'].apply(lambda x:class_name_dict[x])
        combined_data['group_id'] = combined_data['group_id'].apply(lambda x:group_dict[str(x)])
        combined_data.to_excel(eval(save_filename), index = False)
        print(combined_data['predict_label'].value_counts())

In [7]:
def get_server_res(data, url, col_name):
    '''
    服务器接口测试程序
    传入 dict, 传出 DataFrame
    '''
    # data = {'record':[{'id':0,'title':'ss','content':'zzz'},]}
    # data = {"record":marked_human_data.iloc[:5,:3].to_dict(orient = 'records')}
    # url "http://47.93.77.19:10000/correlation_negative"
    headers={'content-type':'application/json'}
    result = requests.post(url,
                      data = json.dumps(data),
                      headers=headers, allow_redirects=True)
    # print(result.text)
    json_data = json.loads(result.text)
    parse_data = []
    elapsed_time = json_data['elapsed_time']
    for i in range(len(json_data['docs'])):
        parse_data.append([json_data['docs'][i]['id'],
                          json_data['docs'][i][col_name]])
    parse_data = pd.DataFrame(parse_data, columns = ['id', col_name])    
    return parse_data, elapsed_time

In [8]:
def get_serve_data(day_list, sql_one_day, url, col_name):
    combined_data = pd.DataFrame()
    for day_select in day_list:
        print('-- day_select: ', day_select)
        mysql_data = pd.read_sql(eval(sql_one_day), engine)
        print('去空值前：', mysql_data.shape)
        mysql_data = mysql_data.drop_duplicates(subset = ['title', 'content'])
        print('去空值后：', mysql_data.shape)
        data = {"record":mysql_data.loc[:,['id', 'title' ,'content']].to_dict(orient = 'records')}
        
        parse_data, elapsed_time = get_server_res(data, url)
        print('elapsed_time: ', elapsed_time)
        
        parse_data.columns = ['id', 'predict_label']
        parse_data['predict_label'] = parse_data['predict_label'].apply(lambda x:class_name_dict[x])
        parse_data['label'] = ''
        combined_cor = pd.merge(parse_data, mysql_data, on = 'id', how = 'inner')
        combined_data = pd.concat([combined_data, combined_cor], axis = 0)

        print(combined_cor['predict_label'].value_counts())
    return combined_data

# 基本信息

In [9]:
label_dic={'监管':1,'行业':2,'产品销售':3,'资本市场':4,'公司内部管理':5,'消费服务':6,'其他相关报道':7,'噪音':8}
class_name_dict = {v: k for k, v in label_dic.items()}
class_name_dict

{1: '监管',
 2: '行业',
 3: '产品销售',
 4: '资本市场',
 5: '公司内部管理',
 6: '消费服务',
 7: '其他相关报道',
 8: '噪音'}

In [10]:
group = '1-新闻，2-论坛，3-博客，4-微博，5-纸媒，6-视频，7-外媒，11-微信，13-新闻客户端，15-推特'
group_dict = dict([x.split('-') for x in group.split('，')])
group_dict

{'1': '新闻',
 '11': '微信',
 '13': '新闻客户端',
 '15': '推特',
 '2': '论坛',
 '3': '博客',
 '4': '微博',
 '5': '纸媒',
 '6': '视频',
 '7': '外媒'}

# 保险业

# 银行业

In [36]:
engine = specific_func.get_engine('cbrc')

NameError: name 'create_engine' is not defined

## mysql 数据

In [42]:
# day_select = '2018-09-09'
day_list = get_day_list('2018-09-08', '2018-09-10')
print(day_list)

['2018-09-09', '2018-09-10']


### 获取数据

In [14]:
# sql_circ_cor_one_day = "select t1.id, t1.publishtime, t1.title,t2.text as content \
#                             from elint_web_docinfo t1, wise_web_docinfo_text t2 \
#                                 where t1.id = t2.doc_id \
#                                   and date_format(t1.publishtime, '%%Y-%%m-%%d') = '{0}'".format('2018-08-07')
# # 实际
# circ_cor = pd.read_sql(sql_circ_cor_one_day, engine)
# print(circ_cor.shape)
# circ_cor.head()

In [43]:
for day_select in day_list:
    print('-- day_select: ', day_select)
    
    # 获取八分类
    sql_one_day = "select t2.urlhash, t1.traffic_id, t2.title as title_1\
                        from wise_web_classify_traffic_docinfo t1, wise_web_docinfo_basic t2 \
                            where t1.base_id=t2.id \
                                  and date_format(t2.publishtime, '%%Y-%%m-%%d') = '{0}' ".format(day_select)
    cbrc_flag = pd.read_sql(sql_one_day, engine)
    print('cbrc_flag：', cbrc_flag.shape)
    
    # 相关数据
    sql_one_day = "select t1.urlhash, t1.title,t2.text as content, t1.group_id, t1.publishtime as publishtime \
                        from elint_web_docinfo t1, wise_web_docinfo_text t2 \
                            where t1.id=t2.doc_id \
                                  and date_format(t1.publishtime, '%%Y-%%m-%%d') = '{0}' \
                                  group by t1.titlehash".format(day_select)
    # titlehash 去重后
    cbrc_cor = pd.read_sql(sql_one_day, engine) 
    print('cbrc_cor：', cbrc_cor.shape)
    
    sql_cbrc_uncor = "select urlhash, title, content, group_id, publishtime \
                            from wise_web_docinfo_uncor \
                            where date_format(publishtime, '%%Y-%%m-%%d') = '{0}'".format(day_select)
    cbrc_uncor = pd.read_sql(sql_cbrc_uncor, engine)  
    print('cbrc_uncor：', cbrc_uncor.shape)

    cbrc_data = pd.concat([cbrc_cor, cbrc_uncor], axis = 0)
    print('去重前：', cbrc_data.shape)
    cbrc_data = cbrc_data.drop_duplicates(subset = ['title', 'content'])
    print('去重后：', cbrc_data.shape)  
    cbrc_data = cbrc_data.dropna(subset = ['title', 'content'], axis = 0)
    print('去空值后：', cbrc_data.shape)  

    cbrc_combined = pd.merge(cbrc_flag, cbrc_data, how = 'inner', on = 'urlhash')
    cbrc_combined['predict_label'] = cbrc_combined['traffic_id'].apply(lambda x:class_name_dict[x])
    cbrc_combined['group_id'] = cbrc_combined['group_id'].apply(lambda x:group_dict[str(x)])
    cbrc_combined['label'] = ''
    cbrc_combined = cbrc_combined[['urlhash', 'predict_label', 'label', 'title', 'content', 'group_id', 'publishtime']]
    fea_filename = 'cbrc_result_class/result/cbrc_class_predict_mysql_%s.xlsx'%day_select
    cbrc_combined.to_excel(fea_filename, index = False)
    print(cbrc_combined.shape)
    print(cbrc_combined['predict_label'].value_counts())
    

-- day_select:  2018-09-09
去重前： (20072, 5)
去重后： (19889, 5)
去空值后： (19889, 5)
(19887, 7)
行业        6061
消费服务      4219
监管        4073
噪音        3718
公司内部管理     714
资本市场       668
产品销售       273
其他相关报道     161
Name: predict_label, dtype: int64
-- day_select:  2018-09-10
去重前： (31001, 5)
去重后： (30714, 5)
去空值后： (30714, 5)
(30690, 7)
行业        10035
监管         7566
消费服务       7053
噪音         3366
公司内部管理     1323
资本市场        944
其他相关报道      227
产品销售        176
Name: predict_label, dtype: int64


### 合并数据

In [44]:
combined_uncor = pd.DataFrame()
for day_select in day_list:
    tmp_data = pd.read_excel('cbrc_result_class/result/cbrc_class_predict_mysql_%s.xlsx'%day_select)
    combined_uncor = pd.concat([combined_uncor, tmp_data], axis = 0)
    
print(combined_uncor.shape)  
print('去重前：', combined_uncor.shape)
combined_uncor = combined_uncor.drop_duplicates(subset = ['title', 'content'])
print('去重后：', combined_uncor.shape)  
combined_uncor = combined_uncor.dropna(subset = ['title', 'content'], axis = 0)
print('去空值后：', combined_uncor.shape)  

print(combined_uncor['predict_label'].value_counts())
combined_uncor.head()

(50577, 7)
去重前： (50577, 7)
去重后： (50327, 7)
去空值后： (50270, 7)
行业        16031
监管        11609
消费服务      11129
噪音         7060
公司内部管理     2027
资本市场       1592
产品销售        442
其他相关报道      380
Name: predict_label, dtype: int64


,urlhash,predict_label,label,title,content,group_id,publishtime
0,-313954450631131328,监管,NaN,潍坊两家保险公司因违法违规被处罚,近日，山东保监局官方网站发布的行政处罚决定中潍坊有2家保险公司被罚款！它们分别是阳光人寿保险...,新闻客户端,2018-09-09 00:00:00
1,5635611554844595200,消费服务,NaN,大家好，我在16年6月办了一张中国银行visa的长城环球旅行卡，我觉得它既是一张储蓄卡也是一...,大家好，我在16年6月办了一张中国银行visa的长城环球旅行卡，我觉得它既是一张储蓄卡也是一...,新闻,2018-09-09 00:00:00
2,8919826005505955840,消费服务,NaN,#滴滴[超话]# http://t,#滴滴[超话]# http://t.cn/RsJxaBs 微信同号13311138121请务...,微博,2018-09-09 00:06:32
3,2343908636206920192,消费服务,NaN,报警服务行业，比较偏门，接触的人，了解的人，太少,报警服务行业，比较偏门，接触的人，了解的人，太少！借此地，给大家普及一下：最早的报警，通俗地...,微博,2018-09-09 00:06:29
4,-6200572542378652672,噪音,NaN,写给2018年的信,关注秦朔朋友圈，ID：qspyq2015 \n \n 这是秦朔朋友圈的第 2202...,微信,2018-09-09 00:09:58


In [45]:
fea_filename = 'cbrc_result_class/result/cbrc_class_predict_mysql_20180911(0909-0910).xlsx'
with pd.ExcelWriter(fea_filename) as writer:
    for label in cbrc_combined['predict_label'].unique():
        tmp_data = cbrc_combined[cbrc_combined['predict_label'] == label]
        if tmp_data.shape[0] > 200:
            N = 200
        else :
            N = tmp_data.shape[0]
        tmp_data.sample(n = N, axis = 0, random_state=42).to_excel(writer,label, index = False)
    
    writer.save()

## 本地模型

## 线上模型

### 计算数据

In [ ]:
# url = "http://192.168.0.104:6001/judge_correlation_yjh"
url = "http://47.93.77.19:6001/judge_correlation_yjh"

day_list = get_day_list('2018-09-08', '2018-09-10')
print(day_list)

In [ ]:
# 不相关
sql_cbrc_uncor = '''"select id, title, content, group_id, publishtime \
                        from wise_web_docinfo_uncor \
                        where date_format(publishtime, '%%Y-%%m-%%d') = '{0}'".format(day_select)'''

save_filename = ''''cbrc_result_class/result/cbrc_uncor_predict_%s.xlsx'%day_select'''
get_serve_data_yjh(day_list, sql_cbrc_uncor, url, 'sec', save_filename)

In [ ]:
# 相关
sql_cbrc_cor = '''
"select t1.id, t1.title,t2.text as content, t1.group_id, t1.publishtime as publishtime \
                    from elint_web_docinfo t1, wise_web_docinfo_text t2 \
                        where t1.id=t2.doc_id \
                              and date_format(t1.publishtime, '%%Y-%%m-%%d') = '{0}' \
                              group by t1.titlehash".format(day_select)
'''

save_filename = ''''cbrc_result_class/result/cbrc_cor_predict_%s.xlsx'%day_select'''
get_serve_data_yjh(day_list, sql_cbrc_cor, url, 'sec', save_filename)

### 合并 & 保存

In [ ]:
combined_uncor = pd.DataFrame()
for day_select in day_list:
    tmp_data = pd.read_excel('cbrc_result_class/result/cbrc_uncor_predict_%s.xlsx'%day_select)
    combined_uncor = pd.concat([combined_uncor, tmp_data], axis = 0)
    
print(combined_uncor.shape)  
print('去重前：', combined_uncor.shape)
combined_uncor = combined_uncor.drop_duplicates(subset = ['title', 'content'])
print('去重后：', combined_uncor.shape)  
combined_uncor = combined_uncor.dropna(subset = ['title', 'content'], axis = 0)
print('去空值后：', combined_uncor.shape)  

print(combined_uncor['predict_label'].value_counts())
combined_uncor.head()

In [ ]:
fea_filename = 'cbrc_result_class/result/cbrc_uncor_predict_class_20180911(0909-0910).xlsx'
with pd.ExcelWriter(fea_filename) as writer:
    for label in combined_uncor['predict_label'].unique():
        tmp_data = combined_uncor[combined_uncor['predict_label'] == label]
        if tmp_data.shape[0] > 100:
            N = 200
            tmp_data.sample(n = N, axis = 0, random_state=42).to_excel(writer,label, index = False)
    
    writer.save()

In [ ]:
combined_cor = pd.DataFrame()
for day_select in day_list:
    tmp_data = pd.read_excel('cbrc_result_class/result/cbrc_cor_predict_%s.xlsx'%day_select)
    combined_cor = pd.concat([combined_cor, tmp_data], axis = 0)

print(combined_cor.shape)  
print('去重前：', combined_cor.shape)
combined_cor = combined_cor.drop_duplicates(subset = ['title', 'content'])
print('去重后：', combined_cor.shape)  
combined_cor = combined_cor.dropna(subset = ['title', 'content'], axis = 0)
print('去空值后：', combined_cor.shape)  

print(combined_cor.shape)  
print(combined_cor['predict_label'].value_counts())
combined_cor.head()

In [ ]:
fea_filename = 'cbrc_result_class/result/cbrc_cor_predict_class_20180911(0909-0910).xlsx'
with pd.ExcelWriter(fea_filename) as writer:
    for label in combined_cor['predict_label'].unique():
        tmp_data = combined_cor[combined_cor['predict_label'] == label]
        if tmp_data.shape[0] > 700:
            N = 200
            tmp_data.sample(n = N, axis = 0, random_state=42).to_excel(writer,label, index = False)
    
    writer.save()

# 保存本文件

In [ ]:
if 0:
    import datetime as dt
    
    def output_HTML(read_file, output_file):
        from nbconvert import HTMLExporter
        import codecs
        import nbformat
        exporter = HTMLExporter()
        # read_file is '.ipynb', output_file is '.html'
        output_notebook = nbformat.read(read_file, as_version=4)
        output, resources = exporter.from_notebook_node(output_notebook)
        codecs.open(output_file, 'w', encoding='utf-8').write(output)

    html_file_folder = 'html_files'
    if not os.path.exists(html_file_folder):
        os.makedirs(html_file_folder)

    today = dt.datetime.now().strftime('%Y%m%d')
    current_file = 'circ_cor_model_2_train.ipynb'
    output_file = 'html_files\%s_%s.html'%(os.path.splitext(current_file)[0], today)
    output_HTML(current_file, output_file)